In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

# 1. 경로 설정
DATA_PATH = "../../data/raw/"
SUB_PATH = "./submissions/"

# 2. 결과물 로드
# m1: 기존 0.78116 모델
# m2: Experiment 12 (AUC 최적화 0.78108)
# m3: Experiment 13 (Optuna 0.78093)
try:
    m1 = pd.read_csv(f"{SUB_PATH}0130-1923.csv")
    m2 = pd.read_csv(f"{SUB_PATH}Pure_0781_AUC_0.77322.csv")
    m3 = pd.read_csv(f"{SUB_PATH}Optuna_Hybrid_0131_2354_AUC_0.7727.csv")
    
    print("모든 파일을 성공적으로 불러왔습니다.")
except FileNotFoundError as e:
    print(f"파일을 찾을 수 없습니다: {e}")
    print("SUB_PATH 내의 파일명을 다시 확인해주세요.")

# 3. 1:1:1 단순 산술 평균 수행
# 세 모델의 'voted' 컬럼 값을 합산하여 3으로 나눕니다.
ensemble_voted = (m1['voted'] + m2['voted'] + m3['voted']) / 3.0

# 4. 제출 파일 생성
submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv")
submission['voted'] = ensemble_voted

# 파일명에 앙상블임을 명시하고 타임스탬프와 평균 점수 기록
timestamp = datetime.now().strftime("%m%d_%H%M")
final_filename = f"{SUB_PATH}Simple_Ensemble_111_{timestamp}.csv"

submission.to_csv(final_filename, index=False)

print("-" * 30)
print(f"최종 앙상블 파일 생성 완료!")
print(f"저장 경로: {final_filename}")
print("-" * 30)

# 5. 간단한 상관관계 확인 (모델들이 서로 얼마나 다른지 확인)
# 상관관계가 0.99 이하로 낮을수록 앙상블 효과가 큽니다.
corr_matrix = pd.DataFrame({
    'm1': m1['voted'],
    'm2': m2['voted'],
    'm3': m3['voted']
}).corr()

print("모델 간 상관관계 행렬:")
print(corr_matrix)

모든 파일을 성공적으로 불러왔습니다.
------------------------------
최종 앙상블 파일 생성 완료!
저장 경로: ./submissions/Simple_Ensemble_111_0201_1510.csv
------------------------------
모델 간 상관관계 행렬:
          m1        m2        m3
m1  1.000000  0.999690  0.999379
m2  0.999690  1.000000  0.999361
m3  0.999379  0.999361  1.000000
